Cellule 1 : Imports et configuration


In [1]:
# ============================================================
# ETL : DATA LAKE (CSV) → DWH
# Tables : DIM_GEOGRAPHIE, DIM_BORNE_RECHARGE
# ============================================================

import os
import pandas as pd
import pyodbc
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from datetime import datetime
from tqdm import tqdm
import io

print("=" * 70)
print("🚀 ETL DATA LAKE CSV → DWH")
print("=" * 70)
print(f"📅 Date d'exécution : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

# Charger les variables d'environnement
load_dotenv()

print("\n✅ Imports réussis")
print("✅ Variables d'environnement chargées")

🚀 ETL DATA LAKE CSV → DWH
📅 Date d'exécution : 2026-02-09 19:37:13

✅ Imports réussis
✅ Variables d'environnement chargées


Cellule 2 : Configuration des connexions


In [2]:
print("\n🔧 CONFIGURATION DES CONNEXIONS")
print("=" * 70)

# ========================================
# Configuration Azure Data Lake
# ========================================
STORAGE_ACCOUNT_NAME = os.getenv("STORAGE_ACCOUNT_NAME")
STORAGE_ACCOUNT_KEY = os.getenv("STORAGE_ACCOUNT_KEY")
CONTAINER_BRONZE = os.getenv("CONTAINER_BRONZE")
CONTAINER_GOLD = os.getenv("CONTAINER_GOLD")

# ========================================
# Configuration DWH
# ========================================
DB_SERVER_DWH = os.getenv("DB_SERVER_DWH")
DB_DATABASE_DWH = os.getenv("DB_DATABASE_DWH")
DB_USERNAME_DWH = os.getenv("DB_USERNAME_DWH")
DB_PASSWORD_DWH = os.getenv("DB_PASSWORD_DWH")

# ========================================
# Chemins des fichiers dans le Data Lake
# ========================================
FICHIER_BORNES = "bornes-irve/2025/01/bornes_irve_hdf_20250131.csv"
FICHIER_COMMUNES = "data_to_BDD_data_gouv_city_france/communes_france_2025/communes_france_2025_20251031.csv"

# ========================================
# Départements Hauts-de-France à filtrer
# ========================================
DEPARTEMENTS_HDF = ['59', '62', '80', '60' , '02']

print(f"✅ Storage Account : {STORAGE_ACCOUNT_NAME}")
print(f"✅ Conteneur Bronze : {CONTAINER_BRONZE}")
print(f"✅ Conteneur Gold : {CONTAINER_GOLD}")
print(f"✅ Serveur DWH : {DB_SERVER_DWH}")
print(f"✅ Base DWH : {DB_DATABASE_DWH}")
print(f"✅ Départements HDF : {', '.join(DEPARTEMENTS_HDF)}")
print("=" * 70)


🔧 CONFIGURATION DES CONNEXIONS
✅ Storage Account : datalakecertifimpe
✅ Conteneur Bronze : bronze-data
✅ Conteneur Gold : data-gouv
✅ Serveur DWH : carter-cash-serveur-student.database.windows.net
✅ Base DWH : DWH_E5_projet_AUTO
✅ Départements HDF : 59, 62, 80, 60, 02


Cellule 3 : Connexion à Azure Data Lake

In [3]:
print("\n🔌 CONNEXION À AZURE DATA LAKE")
print("=" * 70)

try:
    # Créer la chaîne de connexion
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={STORAGE_ACCOUNT_NAME};AccountKey={STORAGE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"
    
    # Créer le client Blob Service
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    
    # Tester la connexion
    blob_service_client.get_service_properties()
    
    print("✅ Connexion réussie à Azure Data Lake")
    
    # Lister les conteneurs
    containers = [container.name for container in blob_service_client.list_containers()]
    print(f"📦 Conteneurs disponibles : {', '.join(containers)}")
    
except Exception as e:
    print(f"❌ ERREUR de connexion au Data Lake : {e}")
    raise

print("=" * 70)


🔌 CONNEXION À AZURE DATA LAKE
❌ ERREUR de connexion au Data Lake : The specified account is disabled.
RequestId:71802d63-301e-008c-04f3-99dadc000000
Time:2026-02-09T18:38:13.9576215Z
ErrorCode:AccountIsDisabled
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AccountIsDisabled</Code><Message>The specified account is disabled.
RequestId:71802d63-301e-008c-04f3-99dadc000000
Time:2026-02-09T18:38:13.9576215Z</Message></Error>


HttpResponseError: The specified account is disabled.
RequestId:71802d63-301e-008c-04f3-99dadc000000
Time:2026-02-09T18:38:13.9576215Z
ErrorCode:AccountIsDisabled
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AccountIsDisabled</Code><Message>The specified account is disabled.
RequestId:71802d63-301e-008c-04f3-99dadc000000
Time:2026-02-09T18:38:13.9576215Z</Message></Error>

Cellule 4 : Connexion au DWH


In [112]:
print("\n🔌 CONNEXION AU DATA WAREHOUSE")
print("=" * 70)

try:
    # Chaîne de connexion SQL Server
    connection_string_dwh = (
        f"DRIVER={{ODBC Driver 18 for SQL Server}};"
        f"SERVER={DB_SERVER_DWH};"
        f"DATABASE={DB_DATABASE_DWH};"
        f"UID={DB_USERNAME_DWH};"
        f"PWD={DB_PASSWORD_DWH};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
        f"Connection Timeout=30;"
    )
    
    cnxn_dwh = pyodbc.connect(connection_string_dwh)
    cursor_dwh = cnxn_dwh.cursor()
    
    # Test de connexion
    cursor_dwh.execute("SELECT @@VERSION")
    version = cursor_dwh.fetchone()[0]
    
    print("✅ Connexion réussie au DWH")
    print(f"📊 Base de données : {DB_DATABASE_DWH}")
    
except Exception as e:
    print(f"❌ ERREUR de connexion au DWH : {e}")
    raise

print("=" * 70)


🔌 CONNEXION AU DATA WAREHOUSE
✅ Connexion réussie au DWH
📊 Base de données : DWH_E5_projet_AUTO


Cellule 5 : Fonction de lecture CSV depuis Data Lake

In [113]:
print("\n📦 FONCTIONS UTILITAIRES")
print("=" * 70)

def lire_csv_depuis_datalake(container_name, blob_path, separator=';', encoding='utf-8'):
    """
    Lit un fichier CSV depuis Azure Data Lake et retourne un DataFrame pandas
    
    Args:
        container_name: Nom du conteneur
        blob_path: Chemin du fichier dans le conteneur
        separator: Séparateur CSV (par défaut ';')
        encoding: Encodage du fichier (par défaut 'utf-8')
    
    Returns:
        DataFrame pandas
    """
    try:
        print(f"⏳ Lecture de {blob_path}...")
        
        # Obtenir le client du conteneur
        container_client = blob_service_client.get_container_client(container_name)
        
        # Télécharger le blob
        blob_client = container_client.get_blob_client(blob_path)
        blob_data = blob_client.download_blob()
        
        # Lire le contenu en mémoire
        content = blob_data.readall()
        
        # Convertir en DataFrame
        df = pd.read_csv(io.BytesIO(content), sep=separator, encoding=encoding, low_memory=False)
        
        print(f"✅ Fichier chargé : {len(df)} lignes, {len(df.columns)} colonnes")
        
        return df
    
    except Exception as e:
        print(f"❌ ERREUR lors de la lecture du fichier {blob_path} : {e}")
        raise

print("✅ Fonctions utilitaires définies")
print("=" * 70)


📦 FONCTIONS UTILITAIRES
✅ Fonctions utilitaires définies


Cellule 6 : Chargement des fichiers CSV


In [114]:
print("\n📥 CHARGEMENT DES FICHIERS CSV DEPUIS DATA LAKE")
print("=" * 70)

# ========================================
# 1. Charger le fichier des bornes IRVE
# ========================================
print("\n1️⃣  Chargement du fichier BORNES IRVE...")
df_bornes = lire_csv_depuis_datalake(CONTAINER_BRONZE, FICHIER_BORNES, separator=';', encoding='utf-8')

print(f"   Colonnes : {list(df_bornes.columns[:10])}...")
print(f"   Premières lignes :")
print(df_bornes.head(2))

# ========================================
# 2. Charger le fichier des communes
# ========================================
print("\n2️⃣  Chargement du fichier COMMUNES...")
df_communes = lire_csv_depuis_datalake(CONTAINER_GOLD, FICHIER_COMMUNES, separator=',', encoding='utf-8')

print(f"   Colonnes : {list(df_communes.columns[:10])}...")
print(f"   Premières lignes :")
print(df_communes.head(2))

print("\n✅ Tous les fichiers CSV chargés avec succès")
print("=" * 70)


📥 CHARGEMENT DES FICHIERS CSV DEPUIS DATA LAKE

1️⃣  Chargement du fichier BORNES IRVE...
⏳ Lecture de bornes-irve/2025/01/bornes_irve_hdf_20250131.csv...
✅ Fichier chargé : 1666 lignes, 22 colonnes
   Colonnes : ['n_amenageur', 'n_operateur', 'n_enseigne', 'id_station', 'n_station', 'ad_station', 'code_insee', 'Xlongitude', 'Ylatitude', 'nbre_pdc']...
   Premières lignes :
                                         n_amenageur  \
0  Communauté d'Agglomération Maubeuge Val de Sambre   
1  Communauté d'Agglomération Maubeuge Val de Sambre   

                     n_operateur            n_enseigne         id_station  \
0  BOUYGUES ENERGIES ET SERVICES  pass pass électrique  FR*H02*P59101*001   
1  BOUYGUES ENERGIES ET SERVICES  pass pass électrique  FR*H02*P59543*001   

                              n_station  \
0      BOUSIGNIES-SUR-ROC - Grand Place   
1  SAINT-RÉMY-DU-NORD - Rue de la Place   

                                 ad_station  code_insee  Xlongitude  \
0      Grand Place 5

Cellule 7 : ETL - Chargement de DIM_GEOGRAPHIE


In [115]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_GEOGRAPHIE")
print("=" * 70)

# ========================================
# FILTRER UNIQUEMENT LES DÉPARTEMENTS HDF (59, 62, 80)
# ========================================
print(f"\n⏳ Filtrage des communes des Hauts-de-France ({', '.join(DEPARTEMENTS_HDF)})...")

df_communes_hdf = df_communes[df_communes['dep_code'].astype(str).isin(DEPARTEMENTS_HDF)].copy()
print(f"✅ {len(df_communes_hdf)} communes trouvées dans les Hauts-de-France")

# ========================================
# PRÉPARATION DES DONNÉES GEOGRAPHIQUES
# ========================================
print("\n⏳ Préparation des données géographiques...")

# Sélectionner et renommer les colonnes nécessaires selon votre schéma de DWH
geo_data = df_communes_hdf[[
    'code_insee',
    'nom_standard',
    'code_postal',
    'reg_code',
    'reg_nom',
    'dep_code',
    'dep_nom',
    'population',
    'superficie_km2',
    'densite',
    'latitude_mairie',
    'longitude_mairie'
]].copy()

geo_data.columns = [
    'Code_INSEE',
    'Nom_Commune',
    'Code_Postal',
    'Reg_Code',
    'Reg_Nom',
    'Dep_Code',
    'Dep_Nom',
    'Population',
    'Superficie_km2',
    'Densite',
    'Latitude',
    'Longitude'
]

# Remplacer les NaN pour éviter les soucis à l'insertion
geo_data = geo_data.fillna({'Code_Postal': '00000', 'Population': 0, 'Superficie_km2': 0.0, 'Densite': 0.0, 'Latitude': 0.0, 'Longitude': 0.0})
geo_data['Population'] = geo_data['Population'].astype(int)

# Supprimer les doublons éventuels sur Code_INSEE
geo_data = geo_data.drop_duplicates(subset=['Code_INSEE'])

print(f"✅ {len(geo_data)} géographies uniques (Code_INSEE) à charger")

# ========================================
# CHARGER LES CODE_INSEE EXISTANTS
# ========================================
print("\n⏳ Chargement des géographies déjà présentes dans le DWH...")
cursor_dwh.execute("SELECT Code_INSEE FROM DIM_GEOGRAPHIE")
geo_existantes = set(row[0] for row in cursor_dwh.fetchall())
print(f"✅ {len(geo_existantes)} géographies déjà présentes dans DIM_GEOGRAPHIE")

# ========================================
# FILTRER LES NOUVELLES GEOGRAPHIES
# ========================================
geo_nouvelles = geo_data[~geo_data['Code_INSEE'].isin(geo_existantes)]
print(f"✅ {len(geo_nouvelles)} nouvelles géographies à insérer")

# ========================================
# INSERTION PAR BATCH (pour efficacité)
# ========================================
if len(geo_nouvelles) > 0:
    insert_query = """
    INSERT INTO DIM_GEOGRAPHIE (
        Code_INSEE, Nom_Commune, Code_Postal, Reg_Code, Reg_Nom,
        Dep_Code, Dep_Nom, Population, Superficie_km2, Densite,
        Latitude, Longitude
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    batch_size = 300
    total_insert = 0
    tuples = [tuple(x) for x in geo_nouvelles.values]
    for i in tqdm(range(0, len(tuples), batch_size), desc="Insertion géographies", unit="batch"):
        batch = tuples[i:i+batch_size]
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            total_insert += len(batch)
        except Exception as e:
            print(f"⚠️ Erreur batch {i//batch_size+1}: {e}")
            for row in batch:
                try:
                    cursor_dwh.execute(insert_query, row)
                    total_insert += 1
                except:
                    pass
            cnxn_dwh.commit()
    print(f"\n✅ {total_insert} lignes insérées dans DIM_GEOGRAPHIE")
else:
    print("✅ Aucune nouvelle géographie à insérer")

# ========================================
# NETTOYAGE DES DOUBLONS
# ========================================
print("\n🧹 Nettoyage des doublons dans DIM_GEOGRAPHIE...")

# Détecter les doublons par code INSEE (si jamais)
cursor_dwh.execute("""
SELECT Code_INSEE, COUNT(*) AS nb
FROM DIM_GEOGRAPHIE
GROUP BY Code_INSEE
HAVING COUNT(*) > 1
""")
doublons = cursor_dwh.fetchall()
doublons_count = sum([row[1]-1 for row in doublons])
if doublons_count > 0:
    print(f"⚠️ {doublons_count} doublons trouvés, suppression...")
    for row in doublons:
        code_insee = row[0]
        cursor_dwh.execute("""
            DELETE FROM DIM_GEOGRAPHIE
            WHERE Code_INSEE = ?
            AND SK_Geographie NOT IN (
                SELECT MIN(SK_Geographie)
                FROM DIM_GEOGRAPHIE
                WHERE Code_INSEE = ?
            )
        """, code_insee, code_insee)
        cnxn_dwh.commit()
    print("✅ Doublons supprimés.")
else:
    print("✅ Aucun doublon détecté.")

# Validation finale
cursor_dwh.execute("SELECT COUNT(*) FROM DIM_GEOGRAPHIE")
nb_final = cursor_dwh.fetchone()[0]
print(f"\n📊 Total final dans DIM_GEOGRAPHIE : {nb_final}")

print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_GEOGRAPHIE

⏳ Filtrage des communes des Hauts-de-France (59, 62, 80, 60, 02)...
✅ 3788 communes trouvées dans les Hauts-de-France

⏳ Préparation des données géographiques...
✅ 3788 géographies uniques (Code_INSEE) à charger

⏳ Chargement des géographies déjà présentes dans le DWH...
✅ 3788 géographies déjà présentes dans DIM_GEOGRAPHIE
✅ 0 nouvelles géographies à insérer
✅ Aucune nouvelle géographie à insérer

🧹 Nettoyage des doublons dans DIM_GEOGRAPHIE...
✅ Aucun doublon détecté.

📊 Total final dans DIM_GEOGRAPHIE : 3788


📦 Cellule 8 : ETL - Chargement de DIM_BORNE_RECHARGE


In [116]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE DIM_BORNE_RECHARGE")
print("=" * 70)

# ========================================
# MAPPING DES COLONNES CSV → DWH
# ========================================
print("⏳ Préparation des données des bornes...")

# Renommer les colonnes du CSV pour correspondre au schéma DWH
colonnes_mapping = {
    'id_station': 'ID_Station',
    'n_station': 'Nom_Station',
    'type_prise': 'Type_Prise',
    'puiss_max': 'Puissance_kW',
    'n_operateur': 'Operateur',
    'acces_recharge': 'Acces',
    'accessibilité': 'Statut',
    'date_maj': 'Date_Mise_Service',
    'code_insee_commune': 'code_insee_commune',
    'Xlongitude': 'Xlongitude',
    'Ylatitude': 'Ylatitude',
    'nbre_pdc': 'nbre_pdc'  # 🆕 NOUVELLE COLONNE
}

df_bornes_clean = df_bornes.rename(columns=colonnes_mapping)

# Sélectionner uniquement les colonnes nécessaires
colonnes_dwh = [
    'ID_Station',
    'Nom_Station',
    'Type_Prise',
    'Puissance_kW',
    'Operateur',
    'Acces',
    'Statut',
    'Date_Mise_Service',
    'code_insee_commune',
    'Xlongitude',
    'Ylatitude',
    'nbre_pdc'  # 🆕 NOUVELLE COLONNE
]

df_bornes_clean = df_bornes_clean[colonnes_dwh].copy()

# ========================================
# NETTOYAGE ET CONVERSION DES TYPES
# ========================================
print("⏳ Nettoyage et conversion des données...")

# Conversion de la puissance en numérique
df_bornes_clean['Puissance_kW'] = pd.to_numeric(df_bornes_clean['Puissance_kW'], errors='coerce').fillna(0.0)

# 🆕 Conversion du nombre de PDC en entier
df_bornes_clean['nbre_pdc'] = pd.to_numeric(df_bornes_clean['nbre_pdc'], errors='coerce').fillna(0).astype(int)

# Conversion de la date
df_bornes_clean['Date_Mise_Service'] = pd.to_datetime(df_bornes_clean['Date_Mise_Service'], errors='coerce')

# Nettoyage des valeurs NULL pour les champs obligatoires
df_bornes_clean['ID_Station'] = df_bornes_clean['ID_Station'].fillna('STATION_INCONNUE')
df_bornes_clean['Nom_Station'] = df_bornes_clean['Nom_Station'].fillna('Non renseigné')
df_bornes_clean['Type_Prise'] = df_bornes_clean['Type_Prise'].fillna('Type inconnu')
df_bornes_clean['Operateur'] = df_bornes_clean['Operateur'].fillna('Opérateur inconnu')
df_bornes_clean['Acces'] = df_bornes_clean['Acces'].fillna('Non renseigné')
df_bornes_clean['Statut'] = df_bornes_clean['Statut'].fillna('Non renseigné')
df_bornes_clean['Xlongitude'] = df_bornes_clean['Xlongitude'].astype(str).fillna('0.0')
df_bornes_clean['Ylatitude'] = df_bornes_clean['Ylatitude'].astype(str).fillna('0.0')

# ========================================
# FORMATAGE DU CODE INSEE (PADDING À 5 CHIFFRES)
# ========================================
print("⏳ Formatage des codes INSEE...")

# Convertir en string et nettoyer
df_bornes_clean['code_insee_commune'] = df_bornes_clean['code_insee_commune'].astype(str).str.strip()

# Remplacer 'nan' ou valeurs vides par '00000'
df_bornes_clean['code_insee_commune'] = df_bornes_clean['code_insee_commune'].replace(['nan', 'None', ''], '00000')

# Ajouter un zéro devant si le code fait 4 chiffres (ex: 2563 -> 02563)
def formater_code_insee(code):
    """Formate le code INSEE pour qu'il ait toujours 5 chiffres"""
    code = str(code).strip()
    
    # Si c'est un code numérique
    if code.isdigit():
        # Padding avec des zéros à gauche pour avoir 5 chiffres
        return code.zfill(5)
    else:
        # Sinon, retourner tel quel ou code par défaut
        return code if code != '' else '00000'

df_bornes_clean['code_insee_commune'] = df_bornes_clean['code_insee_commune'].apply(formater_code_insee)

print(f"✅ Codes INSEE formatés")
print(f"\n📊 Exemples de codes INSEE après formatage :")
print(df_bornes_clean['code_insee_commune'].head(10).tolist())

# Supprimer les doublons sur la combinaison ID_Station + Type_Prise + Puissance
df_bornes_clean = df_bornes_clean.drop_duplicates(
    subset=['ID_Station', 'Type_Prise', 'Puissance_kW']
)

print(f"\n✅ {len(df_bornes_clean)} bornes uniques préparées")

# ========================================
# CHARGER LES BORNES EXISTANTES EN MÉMOIRE
# ========================================
print("\n⏳ Chargement des bornes déjà présentes dans le DWH...")
cursor_dwh.execute("""
SELECT ID_Station, Type_Prise, Puissance_kW
FROM DIM_BORNE_RECHARGE
""")

bornes_existantes = set()
for row in cursor_dwh.fetchall():
    cle = (row[0], row[1], float(row[2]))
    bornes_existantes.add(cle)

print(f"✅ {len(bornes_existantes)} bornes déjà présentes dans le DWH")

# ========================================
# FILTRER LES NOUVELLES BORNES À INSÉRER
# ========================================
print("\n⏳ Filtrage des nouvelles bornes à insérer...")
nouvelles_bornes = []

for idx, row in df_bornes_clean.iterrows():
    cle = (row['ID_Station'], row['Type_Prise'], float(row['Puissance_kW']))
    
    if cle not in bornes_existantes:
        nouvelles_bornes.append((
            row['ID_Station'],
            row['Nom_Station'],
            row['Type_Prise'],
            float(row['Puissance_kW']),
            row['Operateur'],
            row['Acces'],
            row['Statut'],
            row['Date_Mise_Service'],
            row['code_insee_commune'],  # Formaté à 5 chiffres
            row['Xlongitude'],
            row['Ylatitude'],
            int(row['nbre_pdc'])  # 🆕 NOUVELLE COLONNE
        ))

skip_count = len(df_bornes_clean) - len(nouvelles_bornes)
print(f"✅ {len(nouvelles_bornes)} nouvelles bornes à insérer")
print(f"⚠️  {skip_count} bornes déjà existantes (ignorées)")

# ========================================
# INSERTION PAR BATCH
# ========================================
if len(nouvelles_bornes) > 0:
    print("\n⏳ Insertion des nouvelles bornes...")
    
    insert_query = """
    INSERT INTO DIM_BORNE_RECHARGE (
        ID_Station, Nom_Station, Type_Prise, Puissance_kW,
        Operateur, Acces, Statut, Date_Mise_Service,
        code_insee_commune, Xlongitude, Ylatitude, nbre_pdc
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    batch_size = 500
    insert_count = 0
    
    for i in tqdm(range(0, len(nouvelles_bornes), batch_size), desc="Insertion bornes", unit="batch"):
        batch = nouvelles_bornes[i:i+batch_size]
        
        try:
            cursor_dwh.executemany(insert_query, batch)
            cnxn_dwh.commit()
            insert_count += len(batch)
        except Exception as e:
            print(f"\n⚠️  Erreur lors de l'insertion du batch {i//batch_size + 1} : {e}")
            # En cas d'erreur, insérer ligne par ligne pour ce batch
            for borne in batch:
                try:
                    cursor_dwh.execute(insert_query, borne)
                    insert_count += 1
                except Exception as e_detail:
                    print(f"   ❌ Erreur insertion : {e_detail}")
                    print(f"   Données problématiques : INSEE={borne[8]}, nbre_pdc={borne[11]}")
            cnxn_dwh.commit()
    
    print(f"\n✅ Insertion terminée : {insert_count} bornes insérées")
else:
    print("\n✅ Aucune nouvelle borne à insérer")
    insert_count = 0

# ========================================
# NETTOYAGE DES DOUBLONS
# ========================================
print("\n🧹 NETTOYAGE DES DOUBLONS")
print("=" * 70)

print("⏳ Détection des doublons...")
cursor_dwh.execute("""
SELECT ID_Station, Type_Prise, Puissance_kW, COUNT(*) as Nb_Doublons
FROM DIM_BORNE_RECHARGE
GROUP BY ID_Station, Type_Prise, Puissance_kW
HAVING COUNT(*) > 1
""")

doublons = cursor_dwh.fetchall()
nb_groupes_doublons = len(doublons)

if nb_groupes_doublons > 0:
    print(f"⚠️  {nb_groupes_doublons} groupes de doublons trouvés")
    
    total_doublons_supprimes = 0
    
    # Supprimer les doublons pour chaque groupe
    for doublon in tqdm(doublons, desc="Suppression doublons", unit="groupe"):
        id_station = doublon[0]
        type_prise = doublon[1]
        puissance = doublon[2]
        nb_occurrences = doublon[3]
        
        # Récupérer tous les SK_Borne pour ce groupe
        cursor_dwh.execute("""
        SELECT SK_Borne
        FROM DIM_BORNE_RECHARGE
        WHERE ID_Station = ? AND Type_Prise = ? AND Puissance_kW = ?
        ORDER BY SK_Borne ASC
        """, id_station, type_prise, puissance)
        
        sk_bornes = [row[0] for row in cursor_dwh.fetchall()]
        
        # Garder le premier (plus ancien), supprimer les autres
        sk_a_garder = sk_bornes[0]
        sk_a_supprimer = sk_bornes[1:]
        
        # Supprimer les doublons
        for sk in sk_a_supprimer:
            cursor_dwh.execute("""
            DELETE FROM DIM_BORNE_RECHARGE
            WHERE SK_Borne = ?
            """, sk)
            total_doublons_supprimes += 1
    
    cnxn_dwh.commit()
    
    print(f"\n✅ Nettoyage terminé :")
    print(f"   • {nb_groupes_doublons} groupes de doublons traités")
    print(f"   • {total_doublons_supprimes} enregistrements en doublon supprimés")
else:
    print("✅ Aucun doublon trouvé dans DIM_BORNE_RECHARGE")

# ========================================
# VÉRIFICATION FINALE
# ========================================
print("\n🔍 VÉRIFICATION FINALE")
print("=" * 70)

cursor_dwh.execute("SELECT COUNT(*) FROM DIM_BORNE_RECHARGE")
total_final = cursor_dwh.fetchone()[0]
print(f"📊 Nombre total de bornes dans DIM_BORNE_RECHARGE : {total_final}")

# Vérifier qu'il n'y a plus de doublons
cursor_dwh.execute("""
SELECT COUNT(*) 
FROM (
    SELECT ID_Station, Type_Prise, Puissance_kW, COUNT(*) as Nb
    FROM DIM_BORNE_RECHARGE
    GROUP BY ID_Station, Type_Prise, Puissance_kW
    HAVING COUNT(*) > 1
) AS Doublons
""")
nb_doublons_restants = cursor_dwh.fetchone()[0]

if nb_doublons_restants == 0:
    print("✅ Aucun doublon détecté - Table propre !")
else:
    print(f"⚠️  ATTENTION : {nb_doublons_restants} doublons restants détectés")

# Vérification de la répartition par département
print("\n📍 Répartition des bornes par département (code INSEE) :")
cursor_dwh.execute("""
SELECT 
    LEFT(code_insee_commune, 2) as Dep_Code,
    COUNT(*) as Nb_Bornes,
    SUM(nbre_pdc) as Total_PDC
FROM DIM_BORNE_RECHARGE
WHERE code_insee_commune IS NOT NULL
GROUP BY LEFT(code_insee_commune, 2)
ORDER BY Nb_Bornes DESC
""")

for row in cursor_dwh.fetchall():
    print(f"   • Département {row[0]} : {row[1]} bornes - {row[2]} points de charge")

# Statistiques sur les points de charge
print("\n⚡ Statistiques sur les points de charge (PDC) :")
cursor_dwh.execute("""
SELECT 
    COUNT(*) as Nb_Stations,
    SUM(nbre_pdc) as Total_PDC,
    AVG(CAST(nbre_pdc AS FLOAT)) as Moyenne_PDC_Par_Station,
    MIN(nbre_pdc) as Min_PDC,
    MAX(nbre_pdc) as Max_PDC
FROM DIM_BORNE_RECHARGE
""")

stats_pdc = cursor_dwh.fetchone()
print(f"   • Nombre total de stations : {stats_pdc[0]}")
print(f"   • Total points de charge : {stats_pdc[1]}")
print(f"   • Moyenne PDC/station : {stats_pdc[2]:.2f}")
print(f"   • Min PDC : {stats_pdc[3]}")
print(f"   • Max PDC : {stats_pdc[4]}")

# Afficher quelques exemples de bornes insérées
print("\n📍 Exemples de bornes insérées :")
cursor_dwh.execute("""
SELECT TOP 5 
    ID_Station, Nom_Station, Type_Prise, Puissance_kW, 
    code_insee_commune, Xlongitude, Ylatitude, nbre_pdc
FROM DIM_BORNE_RECHARGE
ORDER BY SK_Borne DESC
""")

for row in cursor_dwh.fetchall():
    print(f"   • Station: {row[0]} | {row[1]}")
    print(f"     Type: {row[2]} | Puissance: {row[3]} kW | PDC: {row[7]}")
    print(f"     Commune INSEE: {row[4]} | Coords: ({row[5]}, {row[6]})")

print("\n" + "=" * 70)
print(f"✅ CHARGEMENT TERMINÉ :")
print(f"   • {insert_count} nouvelles bornes insérées")
print(f"   • {skip_count} bornes déjà existantes (ignorées)")
if nb_groupes_doublons > 0:
    print(f"   • {total_doublons_supprimes} doublons supprimés")
print(f"   • {total_final} bornes uniques au total")
print("=" * 70)


📥 ETL - CHARGEMENT DE DIM_BORNE_RECHARGE
⏳ Préparation des données des bornes...
⏳ Nettoyage et conversion des données...
⏳ Formatage des codes INSEE...
✅ Codes INSEE formatés

📊 Exemples de codes INSEE après formatage :
['59101', '59543', '59392', '59225', '59324', '59365', '59514', '59231', '59230', '59406']

✅ 913 bornes uniques préparées

⏳ Chargement des bornes déjà présentes dans le DWH...
✅ 913 bornes déjà présentes dans le DWH

⏳ Filtrage des nouvelles bornes à insérer...
✅ 0 nouvelles bornes à insérer
⚠️  913 bornes déjà existantes (ignorées)

✅ Aucune nouvelle borne à insérer

🧹 NETTOYAGE DES DOUBLONS
⏳ Détection des doublons...
✅ Aucun doublon trouvé dans DIM_BORNE_RECHARGE

🔍 VÉRIFICATION FINALE
📊 Nombre total de bornes dans DIM_BORNE_RECHARGE : 913
✅ Aucun doublon détecté - Table propre !

📍 Répartition des bornes par département (code INSEE) :
   • Département 59 : 396 bornes - 1331 points de charge
   • Département 02 : 211 bornes - 420 points de charge
   • Département

📦 Cellule 09 : ETL - Chargement de FAIT_DISPONIBILITE_BORNE

In [117]:
from tqdm import tqdm

print("\n📥 ETL - CHARGEMENT DE FAIT_DISPONIBILITE_BORNE")
print("=" * 70)

# ========================================
# ÉTAPE 1 : EXTRACTION DES DONNÉES DEPUIS DIM_BORNE_RECHARGE
# ========================================
print("\n⏳ Extraction des données depuis DIM_BORNE_RECHARGE...")

cursor_dwh.execute("""
SELECT 
    SK_Borne,
    code_insee_commune,
    nbre_pdc,
    Puissance_kW,
    Date_Mise_Service
FROM DIM_BORNE_RECHARGE
WHERE Date_Mise_Service IS NOT NULL
    AND code_insee_commune IS NOT NULL
    AND code_insee_commune != '00000'
ORDER BY SK_Borne
""")

bornes_source = cursor_dwh.fetchall()
print(f"✅ {len(bornes_source)} bornes extraites de DIM_BORNE_RECHARGE")

if len(bornes_source) == 0:
    print("⚠️  Aucune borne à traiter. Vérifiez DIM_BORNE_RECHARGE.")
    print("=" * 70)
else:
    # ========================================
    # ÉTAPE 2 : CHARGEMENT DES TABLES DE RÉFÉRENCE EN MÉMOIRE
    # ========================================
    print("\n⏳ Chargement des références DIM_GEOGRAPHIE...")
    cursor_dwh.execute("SELECT Code_INSEE, SK_Geographie FROM DIM_GEOGRAPHIE")
    mapping_geo = {row[0]: row[1] for row in cursor_dwh.fetchall()}
    print(f"✅ {len(mapping_geo)} codes INSEE chargés")

    print("\n⏳ Chargement des références DIM_TEMPS...")
    cursor_dwh.execute("SELECT CAST(Date AS DATE), SK_Temps FROM DIM_TEMPS")
    mapping_temps = {row[0]: row[1] for row in cursor_dwh.fetchall()}
    print(f"✅ {len(mapping_temps)} dates chargées")

    # ========================================
    # ÉTAPE 3 : RÉSOLUTION DES CLÉS ÉTRANGÈRES
    # ========================================
    print("\n⏳ Résolution des clés étrangères (SK_Geographie, SK_Temps)...")
    
    faits_a_inserer = []
    stats_erreurs = {
        'geo_introuvable': 0,
        'temps_introuvable': 0,
        'date_null': 0,
        'valides': 0
    }
    
    for borne in tqdm(bornes_source, desc="Traitement bornes", unit="borne"):
        sk_borne = borne[0]
        code_insee = borne[1]
        nbre_pdc = borne[2] if borne[2] is not None else 0
        puissance_kw = borne[3] if borne[3] is not None else 0.0
        date_mise_service = borne[4]
        
        # Vérification de la date
        if date_mise_service is None:
            stats_erreurs['date_null'] += 1
            continue
        
        # Résoudre SK_Geographie via code INSEE
        sk_geographie = mapping_geo.get(code_insee)
        if sk_geographie is None:
            stats_erreurs['geo_introuvable'] += 1
            continue
        
        # Résoudre SK_Temps via Date_Mise_Service
        # Convertir en date si nécessaire
        if isinstance(date_mise_service, str):
            from datetime import datetime
            date_mise_service = datetime.strptime(date_mise_service, '%Y-%m-%d').date()
        
        sk_temps = mapping_temps.get(date_mise_service)
        if sk_temps is None:
            stats_erreurs['temps_introuvable'] += 1
            continue
        
        # ✅ Tout est OK, on prépare l'insertion
        stats_erreurs['valides'] += 1
        faits_a_inserer.append((
            sk_temps,
            sk_borne,
            sk_geographie,
            int(nbre_pdc),  # Nombre_prise_disponible
            float(puissance_kw),  # Puissance_Totale_kW
            date_mise_service  # Date_MAJ
        ))
    
    # ========================================
    # RAPPORT DE RÉSOLUTION
    # ========================================
    print(f"\n📊 RAPPORT DE RÉSOLUTION DES CLÉS :")
    print(f"   ✅ Lignes valides : {stats_erreurs['valides']}")
    if stats_erreurs['geo_introuvable'] > 0:
        print(f"   ⚠️  Géographie introuvable : {stats_erreurs['geo_introuvable']}")
    if stats_erreurs['temps_introuvable'] > 0:
        print(f"   ⚠️  Date introuvable dans DIM_TEMPS : {stats_erreurs['temps_introuvable']}")
    if stats_erreurs['date_null'] > 0:
        print(f"   ⚠️  Date NULL ignorée : {stats_erreurs['date_null']}")
    
    # ========================================
    # ÉTAPE 4 : CHARGER LES FAITS EXISTANTS
    # ========================================
    print("\n⏳ Chargement des faits déjà présents dans le DWH...")
    cursor_dwh.execute("""
    SELECT SK_Borne, SK_Temps
    FROM FAIT_DISPONIBILITE_BORNE
    """)
    
    faits_existants = set()
    for row in cursor_dwh.fetchall():
        cle = (row[0], row[1])  # (SK_Borne, SK_Temps)
        faits_existants.add(cle)
    
    print(f"✅ {len(faits_existants)} faits déjà présents")
    
    # ========================================
    # ÉTAPE 5 : FILTRER LES NOUVEAUX FAITS
    # ========================================
    print("\n⏳ Filtrage des nouveaux faits à insérer...")
    nouveaux_faits = []
    
    for fait in faits_a_inserer:
        sk_temps = fait[0]
        sk_borne = fait[1]
        cle = (sk_borne, sk_temps)
        
        if cle not in faits_existants:
            nouveaux_faits.append(fait)
    
    skip_count = len(faits_a_inserer) - len(nouveaux_faits)
    print(f"✅ {len(nouveaux_faits)} nouveaux faits à insérer")
    print(f"⚠️  {skip_count} faits déjà existants (ignorés)")
    
    # ========================================
    # ÉTAPE 6 : INSERTION PAR BATCH
    # ========================================
    if len(nouveaux_faits) > 0:
        print("\n⏳ Insertion des nouveaux faits...")
        
        insert_query = """
        INSERT INTO FAIT_DISPONIBILITE_BORNE (
            SK_Temps,
            SK_Borne,
            SK_Geographie,
            Nombre_prise_disponible,
            Puissance_Totale_kW,
            Date_MAJ
        )
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        batch_size = 500
        insert_count = 0
        
        for i in tqdm(range(0, len(nouveaux_faits), batch_size), desc="Insertion faits", unit="batch"):
            batch = nouveaux_faits[i:i+batch_size]
            
            try:
                cursor_dwh.executemany(insert_query, batch)
                cnxn_dwh.commit()
                insert_count += len(batch)
            except Exception as e:
                print(f"\n⚠️  Erreur lors de l'insertion du batch {i//batch_size + 1} : {e}")
                # En cas d'erreur, insérer ligne par ligne
                for fait in batch:
                    try:
                        cursor_dwh.execute(insert_query, fait)
                        insert_count += 1
                    except Exception as e_detail:
                        print(f"   ❌ Erreur insertion : {e_detail}")
                        print(f"   Données : SK_Temps={fait[0]}, SK_Borne={fait[1]}, SK_Geo={fait[2]}")
                cnxn_dwh.commit()
        
        print(f"\n✅ Insertion terminée : {insert_count} faits insérés")
    else:
        print("\n✅ Aucun nouveau fait à insérer")
        insert_count = 0
    
    # ========================================
    # ÉTAPE 7 : NETTOYAGE DES DOUBLONS
    # ========================================
    print("\n🧹 NETTOYAGE DES DOUBLONS")
    print("=" * 70)
    
    print("⏳ Détection des doublons...")
    cursor_dwh.execute("""
    SELECT SK_Borne, SK_Temps, COUNT(*) as Nb_Doublons
    FROM FAIT_DISPONIBILITE_BORNE
    GROUP BY SK_Borne, SK_Temps
    HAVING COUNT(*) > 1
    """)
    
    doublons = cursor_dwh.fetchall()
    nb_groupes_doublons = len(doublons)
    
    if nb_groupes_doublons > 0:
        print(f"⚠️  {nb_groupes_doublons} groupes de doublons trouvés")
        
        total_doublons_supprimes = 0
        
        # Supprimer les doublons (garder 1 seule ligne par combinaison SK_Borne + SK_Temps)
        for doublon in tqdm(doublons, desc="Suppression doublons", unit="groupe"):
            sk_borne = doublon[0]
            sk_temps = doublon[1]
            
            # Supprimer tous sauf le premier (on ne garde qu'une ligne)
            cursor_dwh.execute("""
            DELETE FROM FAIT_DISPONIBILITE_BORNE
            WHERE SK_Borne = ? AND SK_Temps = ?
            AND NOT EXISTS (
                SELECT TOP 1 1
                FROM FAIT_DISPONIBILITE_BORNE f2
                WHERE f2.SK_Borne = FAIT_DISPONIBILITE_BORNE.SK_Borne
                    AND f2.SK_Temps = FAIT_DISPONIBILITE_BORNE.SK_Temps
                ORDER BY f2.SK_Geographie ASC
            )
            """, sk_borne, sk_temps)
            
            total_doublons_supprimes += cursor_dwh.rowcount
        
        cnxn_dwh.commit()
        
        print(f"\n✅ Nettoyage terminé :")
        print(f"   • {nb_groupes_doublons} groupes de doublons traités")
        print(f"   • {total_doublons_supprimes} enregistrements en doublon supprimés")
    else:
        print("✅ Aucun doublon trouvé dans FAIT_DISPONIBILITE_BORNE")
    
    # ========================================
    # ÉTAPE 8 : VÉRIFICATIONS FINALES
    # ========================================
    print("\n🔍 VÉRIFICATION FINALE")
    print("=" * 70)
    
    cursor_dwh.execute("SELECT COUNT(*) FROM FAIT_DISPONIBILITE_BORNE")
    total_final = cursor_dwh.fetchone()[0]
    print(f"📊 Nombre total de lignes dans FAIT_DISPONIBILITE_BORNE : {total_final}")
    
    # Statistiques globales
    print("\n📈 STATISTIQUES GLOBALES :")
    cursor_dwh.execute("""
    SELECT 
        COUNT(DISTINCT SK_Borne) as Nb_Bornes_Uniques,
        COUNT(DISTINCT SK_Geographie) as Nb_Communes_Uniques,
        COUNT(DISTINCT SK_Temps) as Nb_Dates_Uniques,
        SUM(Nombre_prise_disponible) as Total_Prises,
        AVG(CAST(Puissance_Totale_kW AS FLOAT)) as Puissance_Moyenne,
        MIN(Date_MAJ) as Date_Min,
        MAX(Date_MAJ) as Date_Max
    FROM FAIT_DISPONIBILITE_BORNE
    """)
    
    stats = cursor_dwh.fetchone()
    print(f"   • Bornes uniques : {stats[0]}")
    print(f"   • Communes uniques : {stats[1]}")
    print(f"   • Dates uniques : {stats[2]}")
    print(f"   • Total points de charge : {stats[3]}")
    print(f"   • Puissance moyenne : {stats[4]:.2f} kW")
    print(f"   • Période couverte : {stats[5]} → {stats[6]}")
    
    # Répartition par département
    print("\n📍 Répartition par département :")
    cursor_dwh.execute("""
    SELECT 
        g.Dep_Code,
        g.Dep_Nom,
        COUNT(DISTINCT f.SK_Borne) as Nb_Bornes,
        SUM(f.Nombre_prise_disponible) as Total_Prises,
        COUNT(DISTINCT f.SK_Geographie) as Nb_Communes
    FROM FAIT_DISPONIBILITE_BORNE f
    JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
    GROUP BY g.Dep_Code, g.Dep_Nom
    ORDER BY Nb_Bornes DESC
    """)
    
    
    for row in cursor_dwh.fetchall():
        print(f"   • {row[1]} ({row[0]}) : {row[2]} bornes - {row[3]} prises - {row[4]} communes")
    
    # Top 10 des communes avec le plus de prises
    print("\n🏆 Top 10 des communes avec le plus de points de charge :")
    cursor_dwh.execute("""
    SELECT TOP 10
        g.Nom_Commune,
        g.Dep_Nom,
        COUNT(DISTINCT f.SK_Borne) as Nb_Bornes,
        SUM(f.Nombre_prise_disponible) as Total_Prises,
        AVG(CAST(f.Puissance_Totale_kW AS FLOAT)) as Puissance_Moyenne
    FROM FAIT_DISPONIBILITE_BORNE f
    JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
    GROUP BY g.Nom_Commune, g.Dep_Nom
    ORDER BY Total_Prises DESC
    """)
    
    for row in cursor_dwh.fetchall():
        print(f"   • {row[0]} ({row[1]}) : {row[2]} bornes - {row[3]} prises - {row[4]:.2f} kW moy.")
    
    # Vérification de l'intégrité référentielle
    print("\n🔗 VÉRIFICATION DE L'INTÉGRITÉ RÉFÉRENTIELLE :")
    
    # Vérifier les FK vers DIM_BORNE_RECHARGE
    cursor_dwh.execute("""
    SELECT COUNT(*) 
    FROM FAIT_DISPONIBILITE_BORNE f
    WHERE NOT EXISTS (
        SELECT 1 FROM DIM_BORNE_RECHARGE b 
        WHERE b.SK_Borne = f.SK_Borne
    )
    """)
    orphelins_borne = cursor_dwh.fetchone()[0]
    
    # Vérifier les FK vers DIM_GEOGRAPHIE
    cursor_dwh.execute("""
    SELECT COUNT(*) 
    FROM FAIT_DISPONIBILITE_BORNE f
    WHERE NOT EXISTS (
        SELECT 1 FROM DIM_GEOGRAPHIE g 
        WHERE g.SK_Geographie = f.SK_Geographie
    )
    """)
    orphelins_geo = cursor_dwh.fetchone()[0]
    
    # Vérifier les FK vers DIM_TEMPS
    cursor_dwh.execute("""
    SELECT COUNT(*) 
    FROM FAIT_DISPONIBILITE_BORNE f
    WHERE NOT EXISTS (
        SELECT 1 FROM DIM_TEMPS t 
        WHERE t.SK_Temps = f.SK_Temps
    )
    """)
    orphelins_temps = cursor_dwh.fetchone()[0]
    
    if orphelins_borne == 0 and orphelins_geo == 0 and orphelins_temps == 0:
        print("   ✅ Toutes les clés étrangères sont valides")
    else:
        if orphelins_borne > 0:
            print(f"   ⚠️  {orphelins_borne} faits avec SK_Borne orphelin")
        if orphelins_geo > 0:
            print(f"   ⚠️  {orphelins_geo} faits avec SK_Geographie orphelin")
        if orphelins_temps > 0:
            print(f"   ⚠️  {orphelins_temps} faits avec SK_Temps orphelin")
    
    # Vérifier les doublons restants
    cursor_dwh.execute("""
    SELECT COUNT(*) 
    FROM (
        SELECT SK_Borne, SK_Temps, COUNT(*) as Nb
        FROM FAIT_DISPONIBILITE_BORNE
        GROUP BY SK_Borne, SK_Temps
        HAVING COUNT(*) > 1
    ) AS Doublons
    """)
    nb_doublons_restants = cursor_dwh.fetchone()[0]
    
    if nb_doublons_restants == 0:
        print("   ✅ Aucun doublon détecté (clé : SK_Borne + SK_Temps)")
    else:
        print(f"   ⚠️  ATTENTION : {nb_doublons_restants} doublons restants détectés")
    
    # Afficher quelques exemples de faits insérés
    print("\n📋 EXEMPLES DE FAITS INSÉRÉS :")
    cursor_dwh.execute("""
    SELECT TOP 5
        t.Date,
        t.Jour_Semaine,
        b.ID_Station,
        b.Nom_Station,
        g.Nom_Commune,
        g.Dep_Nom,
        f.Nombre_prise_disponible,
        f.Puissance_Totale_kW
    FROM FAIT_DISPONIBILITE_BORNE f
    JOIN DIM_TEMPS t ON f.SK_Temps = t.SK_Temps
    JOIN DIM_BORNE_RECHARGE b ON f.SK_Borne = b.SK_Borne
    JOIN DIM_GEOGRAPHIE g ON f.SK_Geographie = g.SK_Geographie
    ORDER BY f.SK_Borne DESC
    """)
    
    for row in cursor_dwh.fetchall():
        print(f"\n   📅 Date : {row[0]} ({row[1]})")
        print(f"   🔌 Station : {row[2]}")
        print(f"   📍 Localisation : {row[3]} - {row[4]} ({row[5]})")
        print(f"   ⚡ Capacité : {row[6]} prises - {row[7]} kW")
    
    # ========================================
    # RÉSUMÉ FINAL
    # ========================================
    print("\n" + "=" * 70)
    print("✅ CHARGEMENT TERMINÉ - FAIT_DISPONIBILITE_BORNE")
    print("=" * 70)
    print(f"   📊 Total de lignes dans la table : {total_final}")
    print(f"   ➕ Nouvelles lignes insérées : {insert_count}")
    print(f"   ⏭️  Lignes déjà existantes (ignorées) : {skip_count}")
    if nb_groupes_doublons > 0:
        print(f"   🧹 Doublons supprimés : {total_doublons_supprimes}")
    print(f"\n   📌 Résolution des clés :")
    print(f"      • Lignes valides : {stats_erreurs['valides']}")
    if stats_erreurs['geo_introuvable'] > 0:
        print(f"      • Géographies introuvables : {stats_erreurs['geo_introuvable']}")
    if stats_erreurs['temps_introuvable'] > 0:
        print(f"      • Dates introuvables : {stats_erreurs['temps_introuvable']}")
    if stats_erreurs['date_null'] > 0:
        print(f"      • Dates NULL ignorées : {stats_erreurs['date_null']}")
    print("\n   🎯 Couverture :")
    print(f"      • {stats[0]} bornes uniques")
    print(f"      • {stats[1]} communes équipées")
    print(f"      • {stats[3]} points de charge au total")
    print(f"      • Période : {stats[5]} → {stats[6]}")
    print("=" * 70)

print("\n✅ Cellule 9 terminée avec succès !")
    


📥 ETL - CHARGEMENT DE FAIT_DISPONIBILITE_BORNE

⏳ Extraction des données depuis DIM_BORNE_RECHARGE...
✅ 913 bornes extraites de DIM_BORNE_RECHARGE

⏳ Chargement des références DIM_GEOGRAPHIE...
✅ 3788 codes INSEE chargés

⏳ Chargement des références DIM_TEMPS...
✅ 4018 dates chargées

⏳ Résolution des clés étrangères (SK_Geographie, SK_Temps)...


Traitement bornes: 100%|██████████| 913/913 [00:00<00:00, 456641.97borne/s]


📊 RAPPORT DE RÉSOLUTION DES CLÉS :
   ✅ Lignes valides : 685
   ⚠️  Date introuvable dans DIM_TEMPS : 228

⏳ Chargement des faits déjà présents dans le DWH...


✅ 0 faits déjà présents

⏳ Filtrage des nouveaux faits à insérer...
✅ 685 nouveaux faits à insérer
⚠️  0 faits déjà existants (ignorés)

⏳ Insertion des nouveaux faits...


Insertion faits: 100%|██████████| 2/2 [01:54<00:00, 57.19s/batch]



✅ Insertion terminée : 685 faits insérés

🧹 NETTOYAGE DES DOUBLONS
⏳ Détection des doublons...
✅ Aucun doublon trouvé dans FAIT_DISPONIBILITE_BORNE

🔍 VÉRIFICATION FINALE
📊 Nombre total de lignes dans FAIT_DISPONIBILITE_BORNE : 685

📈 STATISTIQUES GLOBALES :
   • Bornes uniques : 685
   • Communes uniques : 292
   • Dates uniques : 9
   • Total points de charge : 2128
   • Puissance moyenne : 23.51 kW
   • Période couverte : 2020-04-03 → 2021-09-07

📍 Répartition par département :
   • Nord (59) : 350 bornes - 1239 prises - 150 communes
   • Pas-de-Calais (62) : 185 bornes - 578 prises - 70 communes
   • Aisne (02) : 143 bornes - 284 prises - 67 communes
   • Oise (60) : 4 bornes - 14 prises - 3 communes
   • Somme (80) : 3 bornes - 13 prises - 2 communes

🏆 Top 10 des communes avec le plus de points de charge :
   • Lezennes (Nord) : 5 bornes - 130 prises - 16.85 kW moy.
   • Lille (Nord) : 13 bornes - 106 prises - 19.81 kW moy.
   • Dechy (Nord) : 19 bornes - 87 prises - 26.11 kW mo

📊 Cellule 10 : Contrôle Qualité des Données du DWH + Fermeture des Connexions

In [118]:
print("\n🔒 FERMETURE DES CONNEXIONS")
print("=" * 70)

# ========================================
# FERMETURE DU CURSEUR DWH
# ========================================
try:
    if 'cursor_dwh' in locals() and cursor_dwh:
        cursor_dwh.close()
        print("✅ Curseur DWH fermé")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture du curseur : {e}")

# ========================================
# FERMETURE DE LA CONNEXION DWH
# ========================================
try:
    if 'cnxn_dwh' in locals() and cnxn_dwh:
        cnxn_dwh.close()
        print("✅ Connexion DWH fermée")
except Exception as e:
    print(f"⚠️  Erreur lors de la fermeture de la connexion : {e}")

print("\n" + "=" * 70)
print("✅ Toutes les connexions ont été fermées avec succès")
print("=" * 70)
print("\n🎉 FIN DU NOTEBOOK - ETL TERMINÉ")
print("\n📊 RÉSUMÉ DES TABLES CHARGÉES :")
print("   • DIM_GEOGRAPHIE")
print("   • DIM_BORNE_RECHARGE")
print("   • DIM_TEMPS")
print("   • FAIT_DISPONIBILITE_BORNE")
print("\n💾 Votre Data Warehouse est maintenant prêt pour l'analyse !")
print("=" * 70)


🔒 FERMETURE DES CONNEXIONS
✅ Curseur DWH fermé
✅ Connexion DWH fermée

✅ Toutes les connexions ont été fermées avec succès

🎉 FIN DU NOTEBOOK - ETL TERMINÉ

📊 RÉSUMÉ DES TABLES CHARGÉES :
   • DIM_GEOGRAPHIE
   • DIM_BORNE_RECHARGE
   • DIM_TEMPS
   • FAIT_DISPONIBILITE_BORNE

💾 Votre Data Warehouse est maintenant prêt pour l'analyse !
